In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from sklearn.metrics import classification_report
from sklearn.preprocessing import OrdinalEncoder
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

import numpy as np
import plotly.express as px
import folium
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from folium import plugins
from sklearn.preprocessing import StandardScaler
from folium.plugins import HeatMap
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import os
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)


2024-05-09 18:58:26.305839: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
train = pd.read_csv("melb_train_set.csv")
test = pd.read_csv("melb_test_set.csv")
total_data = pd.concat([train,test])

In [4]:
#total_data = pd.read_csv('nodes_record_data.csv')
# split the DataFrame into 3 different subsets
# train = total_data[total_data['train_mask'] == True].drop(columns=['train_mask', 'test_mask'])
# test = total_data[total_data['test_mask'] == True].drop(columns=['train_mask', 'test_mask'])


In [5]:
# Encoding the string dataframe (Regionname), (Council) and (Suburb) columns
ordinal_enc = OrdinalEncoder()
total_data['Encoded_Regionname'] = ordinal_enc.fit_transform(total_data[['Regionname']]).astype(int)
total_data['Encoded_Suburb'] = ordinal_enc.fit_transform(total_data[['Suburb']]).astype(int)
total_data['Encoded_Council'] = ordinal_enc.fit_transform(total_data[['CouncilArea']]).astype(int)
# Encoded Regionname Dictionary, I would use this part for analysing the dataset. As you can see below, the Regionname columns are encoded from 0 to 7.
Encoded_Regionname = total_data[["Regionname", "Encoded_Regionname"]].drop_duplicates().sort_values(by='Encoded_Regionname')
# Encoded Suburb Dictionary. As you can see below, the Suburb columns are encoded.
Encoded_Suburb = total_data[["Suburb", "Encoded_Suburb"]].drop_duplicates().sort_values(by='Encoded_Suburb')
# Encoded Council Dictionary. As you can see below, the Council columns are encoded.
Encoded_council = total_data[["CouncilArea", "Encoded_Council"]].drop_duplicates().sort_values(by='Encoded_Council')
suburb_data = total_data["Encoded_Suburb"].copy()
region_data = total_data["Encoded_Regionname"].copy()
council_data = total_data["Encoded_Council"].copy()


council_data = council_data.reset_index().rename(columns={"index":"node_id"})
suburb_data = suburb_data.reset_index().rename(columns={"index":"node_id"})
region_data = region_data.reset_index().rename(columns={"index":"node_id"})

council_data.to_csv('edges_council_data.csv')
suburb_data.to_csv('edges_suburb_data.csv')
region_data.to_csv('edges_region_data.csv')

Encoded_Suburb["Encoded_Suburb"].to_csv('nodes_suburb_data.csv')
Encoded_Regionname["Encoded_Regionname"].to_csv('nodes_region_data.csv')
Encoded_council["Encoded_Council"].to_csv('nodes_council_data.csv')

In [6]:
df_concated=pd.concat([total_data[["hospital_nearby_0_place_id"]].rename(columns={"hospital_nearby_0_place_id":"a"}),total_data[["hospital_nearby_1_place_id"]].rename(columns={"hospital_nearby_1_place_id":"a"}),total_data[["hospital_nearby_2_place_id"]].rename(columns={"hospital_nearby_2_place_id":"a"}),total_data[["hospital_nearby_3_place_id"]].rename(columns={"hospital_nearby_3_place_id":"a"}),total_data[["hospital_nearby_4_place_id"]].rename(columns={"hospital_nearby_4_place_id":"a"})],ignore_index=True)
ordinal_enc = OrdinalEncoder()
res = ordinal_enc.fit_transform(df_concated)
res = pd.DataFrame(res.astype(int))
len_res = int(len(res)/5)
res

,0
0,1413
1,940
2,1212
3,1413
4,1212
...,...
66890,603
66891,-9223372036854775808
66892,-9223372036854775808
66893,905


In [7]:
total_data["hospital_0_unique"] = res[:len_res]
total_data["hospital_1_unique"] = res[len_res:len_res*2].reset_index().drop("index",axis=1)
total_data["hospital_2_unique"] = res[len_res*2:len_res*3].reset_index().drop("index",axis=1)
total_data["hospital_3_unique"] = res[len_res*3:len_res*4].reset_index().drop("index",axis=1)
total_data["hospital_4_unique"] = res[len_res*4:len_res*5].reset_index().drop("index",axis=1)
Encoded_park_0 = total_data[["hospital_nearby_0_geometry_lat","hospital_nearby_0_geometry_lng","hospital_nearby_0_rating", "hospital_nearby_0_user_ratings_total", "hospital_0_unique"]].drop_duplicates().sort_values(by='hospital_0_unique')
Encoded_park_1 = total_data[["hospital_nearby_1_geometry_lat","hospital_nearby_1_geometry_lng","hospital_nearby_1_rating", "hospital_nearby_1_user_ratings_total", "hospital_1_unique"]].drop_duplicates().sort_values(by='hospital_1_unique')
Encoded_park_2 = total_data[["hospital_nearby_2_geometry_lat","hospital_nearby_2_geometry_lng","hospital_nearby_2_rating", "hospital_nearby_2_user_ratings_total", "hospital_2_unique"]].drop_duplicates().sort_values(by='hospital_2_unique')
Encoded_park_3 = total_data[["hospital_nearby_3_geometry_lat","hospital_nearby_3_geometry_lng","hospital_nearby_3_rating", "hospital_nearby_3_user_ratings_total", "hospital_3_unique"]].drop_duplicates().sort_values(by='hospital_3_unique')
Encoded_park_4 = total_data[["hospital_nearby_4_geometry_lat","hospital_nearby_4_geometry_lng","hospital_nearby_4_rating", "hospital_nearby_4_user_ratings_total", "hospital_4_unique"]].drop_duplicates().sort_values(by='hospital_4_unique')

In [8]:
Encoded_park_4

,hospital_nearby_4_geometry_lat,hospital_nearby_4_geometry_lng,hospital_nearby_4_rating,hospital_nearby_4_user_ratings_total,hospital_4_unique
0,NaN,NaN,NaN,NaN,-9223372036854775808
520,-37.817476,145.121016,4.0,45.0,-9223372036854775808
514,-37.909048,145.079508,3.4,43.0,-9223372036854775808
506,-37.868835,144.992314,2.7,139.0,-9223372036854775808
505,-37.771086,144.909400,3.8,78.0,-9223372036854775808
...,...,...,...,...,...
226,-37.854858,145.014346,4.0,4.0,1551
237,-37.967327,145.073380,4.8,406.0,1551
226,-37.883592,145.058533,4.9,525.0,1551
260,-37.789280,145.160134,3.6,14.0,1551


In [9]:
Encoded_park_0.drop(index=8,inplace=True)
Encoded_park_1.drop(index=0,inplace=True)
Encoded_park_2.drop(index=0,inplace=True)
Encoded_park_3.drop(index=0,inplace=True)
Encoded_park_4.drop(index=0,inplace=True)

In [10]:
# Filling the missing rating values with 0
Encoded_park_0["hospital_nearby_0_rating"].fillna((0), inplace=True)
Encoded_park_0["hospital_nearby_0_user_ratings_total"].fillna((0), inplace=True)
Encoded_park_1["hospital_nearby_1_rating"].fillna((0), inplace=True)
Encoded_park_1["hospital_nearby_1_user_ratings_total"].fillna((0), inplace=True)
Encoded_park_2["hospital_nearby_2_rating"].fillna((0), inplace=True)
Encoded_park_2["hospital_nearby_2_user_ratings_total"].fillna((0), inplace=True)
Encoded_park_3["hospital_nearby_3_rating"].fillna((0), inplace=True)
Encoded_park_3["hospital_nearby_3_user_ratings_total"].fillna((0), inplace=True)
Encoded_park_4["hospital_nearby_4_rating"].fillna((0), inplace=True)
Encoded_park_4["hospital_nearby_4_user_ratings_total"].fillna((0), inplace=True)
save0_train = Encoded_park_0.rename(columns={"hospital_nearby_0_place_id":"a","hospital_nearby_0_geometry_lng":"b","hospital_nearby_0_rating":"c", "hospital_nearby_0_user_ratings_total":"d", "hospital_0_unique":"e","hospital_nearby_0_geometry_lat":"f"}).copy()
save1_train = Encoded_park_1.rename(columns={"hospital_nearby_1_place_id":"a","hospital_nearby_1_geometry_lng":"b","hospital_nearby_1_rating":"c", "hospital_nearby_1_user_ratings_total":"d", "hospital_1_unique":"e","hospital_nearby_1_geometry_lat":"f"}).copy()
save2_train = Encoded_park_2.rename(columns={"hospital_nearby_2_place_id":"a","hospital_nearby_2_geometry_lng":"b","hospital_nearby_2_rating":"c", "hospital_nearby_2_user_ratings_total":"d", "hospital_2_unique":"e","hospital_nearby_2_geometry_lat":"f"}).copy()
save3_train = Encoded_park_3.rename(columns={"hospital_nearby_3_place_id":"a","hospital_nearby_3_geometry_lng":"b","hospital_nearby_3_rating":"c", "hospital_nearby_3_user_ratings_total":"d", "hospital_3_unique":"e","hospital_nearby_3_geometry_lat":"f"}).copy()
save4_train = Encoded_park_4.rename(columns={"hospital_nearby_4_place_id":"a","hospital_nearby_4_geometry_lng":"b","hospital_nearby_4_rating":"c", "hospital_nearby_4_user_ratings_total":"d", "hospital_4_unique":"e","hospital_nearby_4_geometry_lat":"f"}).copy()
saveall_df_train = pd.concat([save0_train,save1_train,save2_train,save3_train,save4_train], ignore_index=True).drop_duplicates(subset='e').sort_values(by='e')
saveall_df_train

,f,b,c,d,e
0,-37.863331,145.010190,4.2,107.0,-9223372036854775808
6494,-37.812258,145.014049,5.0,1.0,0
81,-37.666858,144.948038,2.7,78.0,1
82,-37.845559,145.046446,4.7,69.0,2
4621,-37.858417,145.199421,5.0,9.0,3
...,...,...,...,...,...
2257,-37.576923,144.732284,4.8,20.0,1547
2265,-37.700603,144.960244,3.3,63.0,1548
6251,-37.820937,145.022868,4.5,253.0,1549
9385,-37.989709,145.215003,0.0,0.0,1550


In [11]:
saveall_df_train = pd.concat([save0_train,save1_train,save2_train,save3_train,save4_train], ignore_index=True).sort_values(by='e')
saveall_df_train

,f,b,c,d,e
0,-37.863331,145.010190,4.2,107.0,-9223372036854775808
4609,-37.724756,145.028916,3.6,51.0,-9223372036854775808
4610,-37.819472,145.031595,4.6,44.0,-9223372036854775808
4611,-37.861515,144.687028,5.0,5.0,-9223372036854775808
4612,-37.840938,144.940409,3.1,49.0,-9223372036854775808
...,...,...,...,...,...
7933,-37.854858,145.014346,4.0,4.0,1551
7935,-37.754320,144.958482,3.5,185.0,1551
7936,NaN,NaN,0.0,0.0,1551
9389,-37.789280,145.160134,3.6,14.0,1551


In [12]:
scaler = MinMaxScaler(feature_range = (1,10))
saveall_df_train['f'] = scaler.fit_transform(saveall_df_train['f'].values.reshape(-1, 1))
saveall_df_train['b'] = scaler.fit_transform(saveall_df_train['b'].values.reshape(-1, 1))
saveall_df_train['d'] = scaler.fit_transform(saveall_df_train['d'].values.reshape(-1, 1))
# optional
# saveall_df_train['01'] = 0
# saveall_df_train['02'] = 0
# saveall_df_train['03'] = 0
# saveall_df_train['04'] = 0
# saveall_df_train['05'] = 0
# saveall_df_train['06'] = 0
# saveall_df_train['07'] = 0
# saveall_df_train['08'] = 0
# saveall_df_train['09'] = 0
# saveall_df_train['10'] = 0
# saveall_df_train['11'] = 0
# saveall_df_train['12'] = 0
# saveall_df_train['13'] = 0
# saveall_df_train['14'] = 0
# saveall_df_train['15'] = 0
# saveall_df_train['16'] = 0
# saveall_df_train['17'] = 0
# saveall_df_train['18'] = 0
saveall_df_train

,f,b,c,d,e
0,4.638356,5.738495,4.2,1.563817,-9223372036854775808
4609,6.288213,5.892334,3.6,1.268735,-9223372036854775808
4610,5.160537,5.914335,4.6,1.231850,-9223372036854775808
4611,4.659975,3.083691,5.0,1.026347,-9223372036854775808
4612,4.904968,5.165240,3.1,1.258197,-9223372036854775808
...,...,...,...,...,...
7933,4.739228,5.772636,4.0,1.021077,1551
7935,5.936228,5.313709,3.5,1.974824,1551
7936,NaN,NaN,0.0,1.000000,1551
9389,5.519995,6.970296,3.6,1.073770,1551


In [13]:
saveall_df_train['feat'] = saveall_df_train.drop("e",axis=1).astype(str).agg(', '.join, axis=1)
saveall_df_train = saveall_df_train[["e", "feat"]]
saveall_df_train = saveall_df_train.drop_duplicates('e')
saveall_df_train

,e,feat
0,-9223372036854775808,"4.638356312255155, 5.738495026578448, 4.2, 1.5..."
8171,0,"5.246418228943696, 5.7701962751643805, 5.0, 1...."
81,1,"6.977526547034188, 5.2279126088637895, 2.7, 1...."
2448,2,"4.849946800763632, 6.03634168462122, 4.7, 1.36..."
4621,3,"4.696866344706507, 7.293047396744441, 5.0, 1.0..."
...,...,...
9382,1547,"5.954764179154381, 4.519094082608717, 4.7, 1.8..."
4397,1548,"6.575774881308746, 5.32818036634967, 3.3, 1.33..."
9384,1549,"5.14309582648923, 5.842642775696049, 4.5, 2.33..."
9385,1550,"3.1337148227242437, 7.421054260186338, 0.0, 1.0"


In [14]:
saveall_df_train.to_csv("nodes_hospital_data.csv")

In [15]:
park_data_0 = total_data["hospital_0_unique"].copy()
park_data_1 = total_data["hospital_1_unique"].copy()
park_data_2 = total_data["hospital_2_unique"].copy()
park_data_3 = total_data["hospital_3_unique"].copy()
park_data_4 = total_data["hospital_4_unique"].copy()

park_data_0 = park_data_0.reset_index().rename(columns={"index":"node_id"})
park_data_1 = park_data_1.reset_index().rename(columns={"index":"node_id"})
park_data_2 = park_data_2.reset_index().rename(columns={"index":"node_id"})
park_data_3 = park_data_3.reset_index().rename(columns={"index":"node_id"})
park_data_4 = park_data_4.reset_index().rename(columns={"index":"node_id"})

park_data_0 = park_data_0[park_data_0["hospital_0_unique"]>=0]
park_data_1 = park_data_1[park_data_1["hospital_1_unique"]>=0]
park_data_2 = park_data_2[park_data_2["hospital_2_unique"]>=0]
park_data_3 = park_data_3[park_data_3["hospital_3_unique"]>=0]
park_data_4 = park_data_4[park_data_4["hospital_4_unique"]>=0]

park_data_0 = park_data_0.rename(columns={"hospital_0_unique":"dest_id"}).copy()
park_data_1 = park_data_1.rename(columns={"hospital_1_unique":"dest_id"}).copy()
park_data_2 = park_data_2.rename(columns={"hospital_2_unique":"dest_id"}).copy()
park_data_3 = park_data_3.rename(columns={"hospital_3_unique":"dest_id"}).copy()
park_data_4 = park_data_4.rename(columns={"hospital_4_unique":"dest_id"}).copy()

saveall_df = pd.concat([park_data_0,park_data_1,park_data_2,park_data_3,park_data_4], ignore_index=True)
saveall_df

,node_id,dest_id
0,0,1413
1,1,940
2,2,1212
3,3,1413
4,4,1212
...,...,...
45378,1824,1168
45379,1825,1507
45380,1828,336
45381,1831,336


In [16]:
saveall_df.to_csv('edges_hospital_data.csv')
